### PaaS Authentication

The authenticated user must be an Oracle Cloud user.

You must include the Authorization header and use Basic authorization. That is, the header value must begin with Basic followed by a space, and then base-64 encoded <domain.username>:<password>. You should URL encode the string before you base-64 encode it. For the domain portion of the string, use your identity domain. For example, if your identity domain is exampleDomain, you might use exampleDomain.jdoe@example.com:myPassword. (Every Oracle Cloud service belongs to an identity domain. An identity domain is a construct for managing users and roles through Oracle Identity Cloud Service).

In [ ]:
## 
import json
import keyring
import requests
import dataclasses
from requests import Session
from requests.auth import HTTPBasicAuth
from requests.auth import _basic_auth_str
from secrets import opc_username, opc_password

In [ ]:
## 
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
idm_username = f'{idm_domain_name}.{opc_username}'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = r'https://compute.uscom-central-1.oraclecloud.com/authenticate/'
traditional_iaas_username = f'/Compute-{idm_domain_name}/{opc_username}'
idcs_iaas_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# basic_auth_cred = _basic_auth_str(idcs_iaas_username, opc_password)
json_data={"user":idm_domain_name, "password":opc_password}
files = None
params = None
data=None

basic_auth_cred = _basic_auth_str(idm_username, opc_password)

# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json'), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])
headers = dict([('Authorization', basic_auth_cred), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])

print(f'headers: {headers}')
print(f'json_data: {json_data}')
print(f'_basic_auth_str(idm_username={idm_username}, opc_password), basic_auth_cred: {basic_auth_cred}')
# print(f'idm_username: {idm_username}')
# print(f'idcs_iaas_username: {idcs_iaas_username}')
# print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
# print(f'iaas_auth_endpoint: {iaas_auth_endpoint}')




In [ ]:

# method = 'GET'
# headers = dict([('Authorization', domain_auth_token), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])
# data=None
# files = None
# params = None
# request_url = r'https://apicatalog.oraclecloud.com/v1/orgs/oracle-public/apicollections/compute/18.1.2/apis/Instances/canonical'
# print(f'method: {method}, headers: {headers}, URL: {request_url}')

# response = requests.request(method=method, url=request_url, headers=headers, data=data, files=files, params=params)
# print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')

In [ ]:
## 
session = Session()
session.auth = (idm_username, opc_password)
session.headers.update(headers)

In [ ]:
##
request_url = r'https://apicatalog.oraclecloud.com/v1/search?q=Instances'
# response = requests.request(method=method, url=request_url, headers=headers, data=data, files=files, params=params)
response = session.get(url=request_url,
                         json=json_data,
                         data=data,
                         headers=headers,
                         files=files, 
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')


In [ ]:
response_data = response.json()
response_data.keys()
# pp(response_data['data'][0]['links']['api']['href'])
api_metadata_href = response_data['data'][0]['links']['api']['href']
print(f'API metadata located at: {api_metadata_href}')
# response.json()
# response_data['links']
# response_data['links']

In [ ]:
##
request_url = api_metadata_href
# response = requests.request(method=method, url=request_url, headers=headers, data=data, files=files, params=params)
response = session.get(url=request_url,
                         json=json_data,
                         data=data,
                         headers=headers,
                         files=files, 
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')

In [ ]:
response_data = response.json()
response_data.keys()
api_swagger_href = response_data['links']['canonical']['href']
print(f'API Swagger defn located at: {api_swagger_href}')


In [ ]:
##
request_url = api_swagger_href
# response = requests.request(method=method, url=request_url, headers=headers, data=data, files=files, params=params)
response = session.get(url=request_url,
                         json=json_data,
                         data=data,
                         headers=headers,
                         files=files, 
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')

In [ ]:
response_data = response.json()
response_data.keys()
print(f"Swagger API defn for: {response_data['info']['title']}, swagger version: {response_data['swagger']}, API version: {response_data['info']['version']}")
# pp(response_data)

In [ ]:
# request_url = r'https://apicatalog.oraclecloud.com/v1/orgs/oracle-public/apicollections/compute/18.1.2/apis/Instances'
# response = requests.request(method=method, url=request_url, headers=headers, data=data, files=files, params=params)
# print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')

### IaaS (Compute Classic) Authentication



[https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/Authentication.html](https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/Authentication.html)


API calls to Compute Classic require basic authentication (user name and password). You can pass your username and password with every API call or you can pass a valid authentication token. To get a valid authentication token, send an HTTP request to authenticate the user credentials. If the authentication request succeeds, the server returns a cookie containing an authentication token that is valid for 30 minutes. The client making the API calls must include this cookie in the API calls.


Get an authentication cookie from Compute Classic, as shown in the following cURL command example:

```
curl -i -X POST
     -H "Content-Type: application/oracle-compute-v3+json"
     -d '{"user":"/Compute-acme/jack.jones@example.com","password":"ft7)Dvjo"}'
        https://api-z999.compute.us0.oraclecloud.com/authenticate/

```
5. In the response to the POST request, look for the Set-Cookie header, as shown in the following example.

```
Set-Cookie: nimbula=eyJpZGVudGl0eSI6ICJ7XCJyZWFsbVwiOiBcImNvbXB1dGUtdXM2LXoyOFwiLCBcInZhbHVlXCI6IFwie1xcXCJjdXN0b21lclxcXCI6IFxcXCJDb21wdXRlLWFjbWVjY3NcXFwiLCBcXFwicmVhbG1cXFwiOiBcXFwiY29tcHV0ZS11czYtejI4XFxcIiwgXFxcImVudGl0eV90eXBlXFxcIjogXFxcInVzZXJcXFwiLCBcXFwic2Vzc2lvbl9leHBpcmVzXFxcIjogMTQ2MDQ4NjA5Mi44MDM1NiwgXFxcImV4cGlyZXNcXFwiOiAxNDYwNDc3MDkyLjgwMzU5MiwgXFxcInVzZXJcXFwiOiBcXFwiL0NvbXB1dGUtYWNtZWNjcy9zeWxhamEua2FubmFuQG9yYWNsZS5jb21cXFwiLCBcXFwiZ3JvdXBzXFxcIjogW1xcXCIvQ29tcHV0ZS1hY21lY2NzL0NvbXB1dGUuQ29tcHV0ZV9PcGVyYXRpb25zXFxcIiwgXFxcIi9Db21wdXRlLWFjbWVjY3MvQ29tcHV0ZS5Db21wdXRlX01vbml0b3JcXFwiXX1cIiwgXCJzaWduYXR1cmVcIjogXCJRT0xaeUZZdU54SmdjL3FuSk16MDRnNmRWVng2blY5S0JpYm5zeFNCWXJXcVVJVGZmMkZtdjhoTytaVnZwQVdURGpwczRNMHZTc2RocWw3QmM0VGJpSmhFTWVyNFBjVVgvb05qd2VpaUcyaStBeDBPWmc3SDJFSjRITWQ0S1V3eTl6NlYzRHd4eUhwTjdqM0w0eEFUTDUyeVpVQWVQK1diMkdzU1pjMmpTaHZyNi9ibU1CZ1Nyd2M4MUdxdURBMFN6d044V2VneUF1YVk5QTUxZmxaanJBMGVvVUJudmZ6NGxCUVVIZXloYyt0SXZVaDdUcGU2RGwxd3RSeFNGVVlQR0FEQk9xMExGaVd1QlpaU0FTZVcwOHBZcEZ2a2lOZXdPdU9LaU93dFc3VkFtZ3VHT0E1Yk1ibzYvMm5oZEhTWHJhYmtsY000UVE1LzZUMDJlZUpTYVE9PVwifSJ9; Path=/; Max-Age=1800
Note that the Set-Cookie header and value are in a single line. Line breaks are used in this example for readability.
```
6. Store the authentication cookie in an environment variable, as shown in the following example for a Linux host.

Note that the entire command is in a single line. Line breaks are used in this example for readability.
```
export COMPUTE_COOKIE='nimbula=eyJpZGVudGl0eSI6ICJ7XCJyZWFsbVwiOiBcImNvbXB1dGUtdXM2LXoyOFwiLCBcInZhbHVlXCI6IFwie1xcXCJjdXN0b21lclxcXCI6IFxcXCJDb21wdXRlLWFjbWVjY3NcXFwiLCBcXFwicmVhbG1cXFwiOiBcXFwiY29tcHV0ZS11czYtejI4XFxcIiwgXFxcImVudGl0eV90eXBlXFxcIjogXFxcInVzZXJcXFwiLCBcXFwic2Vzc2lvbl9leHBpcmVzXFxcIjogMTQ2MDQ4NjA5Mi44MDM1NiwgXFxcImV4cGlyZXNcXFwiOiAxNDYwNDc3MDkyLjgwMzU5MiwgXFxcInVzZXJcXFwiOiBcXFwiL0NvbXB1dGUtYWNtZWNjcy9zeWxhamEua2FubmFuQG9yYWNsZS5jb21cXFwiLCBcXFwiZ3JvdXBzXFxcIjogW1xcXCIvQ29tcHV0ZS1hY21lY2NzL0NvbXB1dGUuQ29tcHV0ZV9PcGVyYXRpb25zXFxcIiwgXFxcIi9Db21wdXRlLWFjbWVjY3MvQ29tcHV0ZS5Db21wdXRlX01vbml0b3JcXFwiXX1cIiwgXCJzaWduYXR1cmVcIjogXCJRT0xaeUZZdU54SmdjL3FuSk16MDRnNmRWVng2blY5S0JpYm5zeFNCWXJXcVVJVGZmMkZtdjhoTytaVnZwQVdURGpwczRNMHZTc2RocWw3QmM0VGJpSmhFTWVyNFBjVVgvb05qd2VpaUcyaStBeDBPWmc3SDJFSjRITWQ0S1V3eTl6NlYzRHd4eUhwTjdqM0w0eEFUTDUyeVpVQWVQK1diMkdzU1pjMmpTaHZyNi9ibU1CZ1Nyd2M4MUdxdURBMFN6d044V2VneUF1YVk5QTUxZmxaanJBMGVvVUJudmZ6NGxCUVVIZXloYyt0SXZVaDdUcGU2RGwxd3RSeFNGVVlQR0FEQk9xMExGaVd1QlpaU0FTZVcwOHBZcEZ2a2lOZXdPdU9LaU93dFc3VkFtZ3VHT0E1Yk1ibzYvMm5oZEhTWHJhYmtsY000UVE1LzZUMDJlZUpTYVE9PVwifSJ9; Path=/; Max-Age=1800'
After getting an authentication cookie, you can perform operations on Compute Classic objects.
```



#### How to Generate an Authentication Token in OCI Compute Classic

[https://simplesassim.wordpress.com/2017/06/07/how-to-generate-an-authentication-token-in-oci-compute-classic/](https://simplesassim.wordpress.com/2017/06/07/how-to-generate-an-authentication-token-in-oci-compute-classic/)

```
import requests
 
res = requests.post('https://host/authenticate/', json = { 'user': '/sampleService-sampleIdentityDomain/sampleUser', 'password': 'samplePassword' })
 
if res.no_content:
  print(res.headers['Set-Cookie'])![image.png](attachment:image.png)
  
  
```  

In [ ]:
import keyring
import requests
from requests import Session
from requests.auth import HTTPBasicAuth
from requests.auth import _basic_auth_str
from secrets import opc_username, opc_password

In [ ]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

### Username/pass setup
traditional_iaas_username = f'/Compute-{idm_domain_name}/{opc_username}'

idcs_iaas_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
username = traditional_iaas_username
# username = idcs_iaas_username

json_data={"user":username, "password":opc_password}
basic_auth_cred = _basic_auth_str(username, opc_password)


print(f'basic_auth_cred: {basic_auth_cred}')
print(f'traditional_iaas_username: {traditional_iaas_username}\n')
print(f'idcs_iaas_username: {idcs_iaas_username}\n')
print(f'username: {username}\n')

### END Username/pass setup


files = None
params = None
# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json'), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])
headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json')])
print(f'headers: {headers}')
print(f'json_data: {json_data}')

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}')





In [ ]:
session = Session()
session.auth = (username, opc_password)
session.headers.update(headers)
session.cookies

print(f'session.headers: {session.headers}\n')
print(f'session.auth: {session.auth}\n')
print(f'session.cookies: {session.cookies}\n')


In [ ]:
response = session.post(url=iaas_auth_endpoint,
                         json=json_data,
                         files=files, 
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

In [ ]:
print(f'session.headers: {session.headers}\n')
print(f'session.auth: {session.auth}\n')
print(f'session.cookies: {session.cookies}\n')

In [ ]:
session.cookies.get('nimbula')

In [ ]:
# instances_endpoint = f'{iaas_rest_endpoint}/instance{traditional_iaas_username}'
# instances_endpoint = f'{iaas_rest_endpoint}/instance{idcs_iaas_username}'
instances_endpoint = f'{iaas_rest_endpoint}/instance/'

container = f'{username}/'
# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json'), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])
# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json')])
# headers = dict([('Authorization', basic_auth_cred), 
#                 ('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Cookie', f"nimbula={session.cookies.get('nimbula')}")])

headers.update({'Cookie':f"nimbula={session.cookies.get('nimbula')}"})
session.headers.update({'Cookie':f"nimbula={session.cookies.get('nimbula')}"})


data={"container":username, "password":opc_password}
json_data={"container":username, "password":opc_password}


print(f'instances_endpoint: {instances_endpoint}')
print(f'json_data: {json_data}')
print(f'headers: {headers}\n')
print(f'session.cookies: {session.cookies}\n')


In [ ]:
instances_endpoint = f'{iaas_rest_endpoint}/instance/'

response = session.get(url=instances_endpoint,
                         json=json_data,
                         files=files, 
                         headers=headers,
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

In [ ]:
response.text

In [ ]:
print(f'headers: {headers}\n')
print(f'session.cookies: {session.cookies}\n')
print(f'session.headers: {session.headers}\n')


### IaaS (Compute Classic) REST Calls, Fresh start







In [ ]:
import keyring
import requests
from requests import Session
from requests.auth import HTTPBasicAuth
from requests.auth import _basic_auth_str
from secrets import opc_username, opc_password

In [ ]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')



### Username/pass setup
traditional_iaas_username = f'/Compute-{idm_domain_name}/{opc_username}'
idcs_iaas_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idcs_iaas_username
basic_auth_cred = _basic_auth_str(username, opc_password)

print(f'traditional_iaas_username: {traditional_iaas_username}')
print(f'idcs_iaas_username: {idcs_iaas_username}')
print(f'username: {username}')
print(f'basic_auth_cred: {basic_auth_cred}')

### END Username/pass setup


json_data={"user":username, "password":opc_password}
print(f'\njson_data: {json_data}')


files = None
params = None
# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json'), ('X-ID-TENANT-NAME', 'gc30003'), ('X-PSM-CLI-REQUEST', 'cli'), ('X-PSM-CLI-VERSION', '1.1.20')])
# headers = dict([('Authorization', basic_auth_cred), ('Content-Type', 'application/oracle-compute-v3+json')])
headers = dict([('Content-Type', 'application/oracle-compute-v3+json')])

print(f'headers: {headers}')


In [ ]:
session = Session()
session.auth = (username, opc_password)
session.headers.update(headers)

print(f'session.headers: {session.headers}\n')
print(f'session.auth: {session.auth}\n')
print(f'session.cookies: {session.cookies}\n')

In [ ]:
response = session.post(url=iaas_auth_endpoint, json=json_data)

print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"\nSession Cookies: {session.cookies}")
    print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

In [ ]:
print(f"session.headers before update: {session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
print(f"cookie_header: {cookie_header}\n")
session.headers.update(cookie_header)
print(f"session.headers after update: {session.headers}")


##### Retrieve Names of Containers

https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/op-instance--get.html

In [ ]:


print(f'session.headers: {session.headers}\n')
print(f'session.auth: {session.auth}\n')
print(f'session.cookies: {session.cookies}\n')

print(f'json_data: {json_data}')
print(f'headers: {headers}')


In [ ]:
rest_endpoint = f'{iaas_rest_endpoint}/instance/'
print(f'rest_endpoint: {rest_endpoint}')

response = session.get(url=rest_endpoint)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')
    print(response.text)
    pp(response.headers)

In [ ]:
print(f'instances_endpoint: {instances_endpoint}\n')

print(f'session.headers: {session.headers}\n')
print(f'session.auth: {session.auth}\n')
print(f'session.cookies: {session.cookies}\n')

print(f'json_data: {json_data}')
print(f'headers: {headers}')


In [ ]:
session.headers.update(json_data)

In [ ]:
print(f'session.headers: {session.headers}\n')


In [ ]:
response = session.get(url=instances_endpoint,
                         files=files, 
                         params=params)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

In [ ]:
print(f'session.headers: {session.headers}\n')
print(f'session.cookies: {session.cookies}\n')


###  Final try for Compute Classic?

In [1]:
import keyring
import requests
from json import load, loads, dump, dumps
from requests import Session
from secrets import opc_username, opc_password

In [2]:
idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')



iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [3]:
### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)

print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')
# print(f'basic_auth_cred: {basic_auth_cred}')

### END Username/pass setup
json_data={"user":username, "password":opc_password}
print(f'\njson_data: {json_data}')


files = None
params = None

### https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/SendRequests.html
### Supported Headers shown here
# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                ('Accept', 'application/oracle-compute-v3+directory+json'),
#                ('Accept-Encoding', 'gzip;q=1.0, identity; q=0.5'),
#                ('Content-Encoding', 'deflate'),
#                ('Cookie', '<Set from /authenticate>')
#                ])


headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
               ('Accept', 'application/oracle-compute-v3+directory+json'),
               ])

print(f'headers: {headers}')


idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com

json_data: {'user': '/Compute-587626604/eric.harris@oracle.com', 'password': 'P@ll4dium!'}
headers: {'Content-Type': 'application/oracle-compute-v3+json', 'Accept': 'application/oracle-compute-v3+directory+json'}


In [4]:
session = Session()
session.headers.update(headers)

In [5]:
response = session.post(url=iaas_auth_endpoint, json=json_data)

print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"\nSession Cookies: {session.cookies}")
    print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

Response OK: True, Status Code: 204, URL: https://compute.uscom-central-1.oraclecloud.com/authenticate/
Auth request succeess.


Session Cookies: <RequestsCookieJar[<Cookie nimbula=eyJpZGVudGl0eSI6ICJ7XCJyZWFsbVwiOiBcInVzY29tLWNlbnRyYWwtMVwiLCBcInZhbHVlXCI6IFwie1xcXCJjdXN0b21lclxcXCI6IFxcXCJDb21wdXRlLTU4NzYyNjYwNFxcXCIsIFxcXCJyZWFsbVxcXCI6IFxcXCJ1c2NvbS1jZW50cmFsLTFcXFwiLCBcXFwiZW50aXR5X3R5cGVcXFwiOiBcXFwidXNlclxcXCIsIFxcXCJzZXNzaW9uX2V4cGlyZXNcXFwiOiAxNTIzNTkzMDI1LjcyMjIzMzEsIFxcXCJleHBpcmVzXFxcIjogMTUyMzU4NDAyNS43MjIyNywgXFxcInVzZXJcXFwiOiBcXFwiL0NvbXB1dGUtNTg3NjI2NjA0L2VyaWMuaGFycmlzQG9yYWNsZS5jb21cXFwiLCBcXFwiZ3JvdXBzXFxcIjogW1xcXCIvQ29tcHV0ZS01ODc2MjY2MDQvQ29tcHV0ZS5Db21wdXRlX09wZXJhdGlvbnNcXFwiLCBcXFwiL0NvbXB1dGUtNTg3NjI2NjA0L0NvbXB1dGUuQ29tcHV0ZV9Nb25pdG9yXFxcIl19XCIsIFwic2lnbmF0dXJlXCI6IFwiVzNZUlBaL0YyRitDVGNnUTJmMFdrZk5PTnFBYkhKNnBIUHZ4K2V6a2RjQVp4RXVyQWJMb1VQdnVoUURRVC9CakMvekdmRTFKUkV3VlVoV0RzUXhjUjd6a1hURGpQK3dkR1UxK2NFejRmU1pZUU9OcHdFNmpsRS8rcU01dnIzZTFrTTlDUlk2czhmQjQvVFp

In [6]:
print(f"session.headers before update: {session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
print(f"cookie_header: {cookie_header}\n")
session.headers.update(cookie_header)
print(f"session.headers after update: {session.headers}")

session.headers before update: {'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/oracle-compute-v3+directory+json', 'Connection': 'keep-alive', 'Content-Type': 'application/oracle-compute-v3+json'}

cookie_header: {'Cookie': 'nimbula=eyJpZGVudGl0eSI6ICJ7XCJyZWFsbVwiOiBcInVzY29tLWNlbnRyYWwtMVwiLCBcInZhbHVlXCI6IFwie1xcXCJjdXN0b21lclxcXCI6IFxcXCJDb21wdXRlLTU4NzYyNjYwNFxcXCIsIFxcXCJyZWFsbVxcXCI6IFxcXCJ1c2NvbS1jZW50cmFsLTFcXFwiLCBcXFwiZW50aXR5X3R5cGVcXFwiOiBcXFwidXNlclxcXCIsIFxcXCJzZXNzaW9uX2V4cGlyZXNcXFwiOiAxNTIzNTkzMDI1LjcyMjIzMzEsIFxcXCJleHBpcmVzXFxcIjogMTUyMzU4NDAyNS43MjIyNywgXFxcInVzZXJcXFwiOiBcXFwiL0NvbXB1dGUtNTg3NjI2NjA0L2VyaWMuaGFycmlzQG9yYWNsZS5jb21cXFwiLCBcXFwiZ3JvdXBzXFxcIjogW1xcXCIvQ29tcHV0ZS01ODc2MjY2MDQvQ29tcHV0ZS5Db21wdXRlX09wZXJhdGlvbnNcXFwiLCBcXFwiL0NvbXB1dGUtNTg3NjI2NjA0L0NvbXB1dGUuQ29tcHV0ZV9Nb25pdG9yXFxcIl19XCIsIFwic2lnbmF0dXJlXCI6IFwiVzNZUlBaL0YyRitDVGNnUTJmMFdrZk5PTnFBYkhKNnBIUHZ4K2V6a2RjQVp4RXVyQWJMb1VQdnVoUURRVC9CakMv

In [7]:
rest_endpoint = f'{iaas_rest_endpoint}/instance{idm_service_instance_username}/'
print(f'rest_endpoint: {rest_endpoint}')

rest_endpoint: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/


##### Retrieve Names of all Instances and Subcontainers in a Container

[https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/op-instance-%7Bcontainer%7D-get.html#request](https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/op-instance-%7Bcontainer%7D-get.html#request)

** /instance/{container} **

Retrieves the names of objects and subcontainers that you can access in the specified container.




In [ ]:
response = session.get(url=rest_endpoint)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")
    result = response.json()
    print(f"\n{result}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')
    print(response.text)
    pp(response.headers)

##### Retrieve Details of an Instance

[https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/op-instance-%7Bname%7D-get.html](https://docs.oracle.com/en/cloud/iaas/compute-iaas-cloud/stcsa/op-instance-%7Bname%7D-get.html)

** /instance/{name} **

Retrieves details of the specified instance.



In [ ]:
rest_endpoint = f"{iaas_rest_endpoint}/instance{result['result'][0]}"
print(f'rest_endpoint: {rest_endpoint}')

In [ ]:
response = session.get(url=rest_endpoint)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"Session Cookies: {session.cookies}")
    result = response.json()
    print(f"\n{result}")

else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')
    print(response.text)
    pp(response.headers)